In [20]:
from qiskit import BasicAer, Aer, IBMQ
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.components.initial_states import Zero
from qiskit.aqua.components.optimizers import COBYLA, L_BFGS_B, SLSQP, SPSA
from qiskit.aqua.components.variational_forms import RY, RYRZ, SwapRZ
from qiskit.aqua.operators import WeightedPauliOperator, Z2Symmetries
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock

from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import QuantumError, ReadoutError
from qiskit.providers.aer.noise.errors import pauli_error
from qiskit.providers.aer.noise.errors import depolarizing_error
from qiskit.providers.aer.noise.errors import thermal_relaxation_error

from qiskit import * 

from qiskit.providers.aer import noise
IBMQ.save_account('d3db0c8bd2b2d40d8aa8549ccb819fde27093100d057e34bede25a0d64c91ed50edbbec60102a426f041f6255fe3e39e10a9acf6b7f39a09ee19f2a9cb5248b0')
provider = IBMQ.load_account()


import numpy as np
import matplotlib.pyplot as plt
from functools import partial

configrc.store_credentials:WARNING:2020-08-06 21:53:12,452: Credentials already present. Set overwrite=True to overwrite.
ibmqfactory.load_account:WARNING:2020-08-06 21:53:13,743: Credentials are already in use. The existing account in the session will be replaced.


In [21]:
driver = PySCFDriver(atom='Li .0 .0 .0; H .0 .0 ' + str(1.6), unit=UnitsType.ANGSTROM, charge=0, spin=0, basis='sto3g')
freeze_list = [0]
remove_list = [-3, -2] 

In [22]:
def exact_solver(qubitOp):
    ee = ExactEigensolver(qubitOp)
    result = ee.run()
    ref = result['energy']
    print('Reference value: {}'.format(ref))
    return ref

In [23]:
backend = BasicAer.get_backend('statevector_simulator')

In [24]:
chip_name = 'ibmq_essex'
device = provider.get_backend(chip_name)
coupling_map = device.configuration().coupling_map
noise_model = noise.device.basic_device_noise_model(device.properties())
basis_gates = noise_model.basis_gates

/home/debdeep/anaconda3/lib/python3.7/site-packages/qiskit/providers/aer/noise/device/basic_device_model.py:115: DeprecationWarning: This function is been deprecated and moved to a method of the`NoiseModel` class. For equivalent functionality use `NoiseModel.from_backend(properties, **kwargs).
  DeprecationWarning)


In [25]:
def exact_solver(qubitOp):
    ee = ExactEigensolver(qubitOp)
    result = ee.run()
    ref = result['energy']
    print('Reference value: {}'.format(ref))
    return ref
def compute_LiH_qubitOp(map_type, inter_dist, basis='sto3g'):
    
    distances = np.arrange(0.5,4.0,0.1)
    exact_energy = list()
    vqe_energy = list()
    count = 0
    backend = BasicAer.get_backend('statevector_simulator')
    quantum_instance = QuantumInstance(backend=backend,shots=1024)
    optimizer = COBYLA(maxiter=1000)
    
    for dist in distances:
        qubitOp, num_spin_orbitals, num_particles, qubit_reduction, energy_shift, nre = compute_LiH_qubitOp('parity',dist)
        exact_result = np.real(NumPyEigensolver(qubitOp).run().eigenvalues)
        exact_energies.append(np.real(exact_result[-1]) + energy_shift + nre)
    
        initial_state = HartreeFock(
            num_spin_orbitals,
            num_particles,
            qubit_mapping='parity',
            two_qubit_reduction=qubit_reduction
        ) 
        var_form = UCCSD(
            num_orbitals=num_spin_orbitals,
            num_particles=num_particles,
            initial_state=initial_state,
            qubit_mapping='parity',
            two_qubit_reduction=qubit_reduction
        )
        vqe = VQE(qubitOp, var_form, optimizer)
        vqe_result = np.real(vqe.run(backend)['eigenvalue'] + shift)
        vqe_energies.append(vqe_.run(quantum_instance).eigenvalue.real + energy_shift + nre)
        count = count +1
        print(count)
        

    plt.plot(distances,exact_energy,label='Exact Energy')
    plt.plot(distances,vqe_energy,label='VQE Energy')
    plt.xlabel('Atomic distance (Angstrom)')
    plt.ylabel('Energy')
    plt.legend()
    plt.show()